In [7]:
# !pip install "cloud-sql-python-connector[pg8000]"
# !pip install psycopg2

In [1]:
import psycopg2
from psycopg2 import extras
import pandas as pd

In [2]:
# Replace these with your actual details
user="xxxxx"
password="xxxxx"
database = "cfpb"
host = "xx.xx.xx.xx"

# Connect to your postgres server
conn = psycopg2.connect(
    dbname=database,
    user=user,
    password=password,
    host=host
)
# Create a cursor object
cur = conn.cursor()

In [3]:
# A quick sanity check

# Execute a query
cur.execute("SELECT NOW()")
# Fetch the result
result = cur.fetchone()
print(f"Current time: {result[0]}")

Current time: 2023-07-30 03:18:15.661849+00:00


In [4]:
cfpb = pd.read_csv("complaints.csv")

C:\Users\JYM\AppData\Local\Temp/ipykernel_65400/377903120.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  cfpb = pd.read_csv("complaints.csv")


In [5]:
cfpb.shape

(3895198, 18)

In [6]:
# Convert 'Date_received' and 'Date_sent_to_company' from 'mm/dd/yy' to 'yyyy-mm-dd'
def convert_date_format(date):
    try:
        return pd.to_datetime(date, format='%m/%d/%y').strftime('%Y-%m-%d')
    except ValueError:
        return date  # already in the correct format

cfpb['Date received'] = cfpb['Date received'].apply(convert_date_format)
cfpb['Date sent to company'] = cfpb['Date sent to company'].apply(convert_date_format)
cfpb['Complaint ID'].fillna(0, inplace=True)
cfpb['Complaint ID'] = cfpb['Complaint ID'].astype(float)

In [7]:
cfpb.sample(10)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
2156166,2022-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,33483,NaN,Consent not provided,Web,2022-09-19,Closed with non-monetary relief,Yes,NaN,5997004.0
2049497,2022-09-21,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,17701,NaN,Consent not provided,Web,2022-09-21,Closed with non-monetary relief,Yes,NaN,6003580.0
675469,2022-07-24,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,NaN,NaN,"EQUIFAX, INC.",VA,23139,NaN,Consent not provided,Web,2022-07-24,Closed with explanation,Yes,NaN,5804409.0
8168,2023-07-10,Checking or savings account,Checking account,Managing an account,Deposits and withdrawals,NaN,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,GA,30132,NaN,NaN,Phone,2023-07-10,Closed with explanation,Yes,NaN,7231758.0
2566762,2020-01-30,Student loan,Federal student loan servicing,Dealing with your lender or servicer,Need information about your loan balance or lo...,NaN,NaN,AES/PHEAA,CO,80239,NaN,Consent not provided,Web,2020-01-30,Closed with non-monetary relief,Yes,NaN,3515422.0
1357551,2023-03-29,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,CAPITAL ONE FINANCIAL CORPORATION,OH,43201,NaN,Consent not provided,Web,2023-03-29,Closed with non-monetary relief,Yes,NaN,6762293.0
2127973,2023-06-18,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Investigation took more than 30 days,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",IL,60644,NaN,Consent not provided,Web,2023-06-18,Closed with non-monetary relief,Yes,NaN,7132532.0
1985465,2014-06-27,Debt collection,Payday loan,Improper contact or sharing of info,Talked to a third party about my debt,NaN,NaN,"Enhanced Acquisitions, LLC",MI,49385,NaN,NaN,Web,2014-06-27,Closed with explanation,Yes,No,914620.0
3873201,2023-01-19,Debt collection,Other debt,Written notification about debt,Didn't receive enough information to verify debt,NaN,NaN,"HW Holding, Inc",NC,28625,NaN,Consent not provided,Web,2023-01-19,Closed with explanation,Yes,NaN,6459653.0
1452010,2020-09-04,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,MD,21042,NaN,Consent not provided,Web,2020-09-04,Closed with explanation,Yes,NaN,3829897.0


In [8]:
# create a dictionary to map old column names to new column names
rename_dict = {
    'Date received': 'Date_received',
    'Product': 'Product',
    'Sub-product': 'Sub_product',
    'Issue': 'Issue',
    'Sub-issue': 'Sub_issue',
    'Consumer complaint narrative': 'Consumer_complaint_narrative',
    'Company public response': 'Company_public_response',
    'Company': 'Company',
    'State': 'State',
    'ZIP code': 'ZIP_code',
    'Tags': 'Tags',
    'Consumer consent provided?': 'Consumer_consent_provided',
    'Submitted via': 'Submitted_via',
    'Date sent to company': 'Date_sent_to_company',
    'Company response to consumer': 'Company_response_to_consumer',
    'Timely response?': 'Timely_response',
    'Consumer disputed?': 'Consumer_disputed',
    'Complaint ID': 'Complaint_ID'
}

# rename the columns
cfpb.rename(columns=rename_dict, inplace=True)
cfpb.sample(10)

,Date_received,Product,Sub_product,Issue,Sub_issue,Consumer_complaint_narrative,Company_public_response,Company,State,ZIP_code,Tags,Consumer_consent_provided,Submitted_via,Date_sent_to_company,Company_response_to_consumer,Timely_response,Consumer_disputed,Complaint_ID
2738308,2021-02-16,Vehicle loan or lease,Loan,Struggling to pay your loan,Lender trying to repossess or disable the vehicle,NaN,Company has responded to the consumer and the ...,CREDIT ACCEPTANCE CORPORATION,VA,22505,NaN,Consent not provided,Web,2021-02-16,Closed with explanation,Yes,NaN,4143768.0
2415708,2020-11-13,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Personal information incorrect,XXXX customer service would not remove the in...,NaN,"CBC Companies, Inc.",IN,46112,NaN,Consent provided,Web,2020-11-24,Closed with explanation,Yes,NaN,3953115.0
923750,2016-01-07,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,RUSHMORE LOAN MANAGEMENT SERVICES LLC,FL,33062,NaN,NaN,Referral,2016-02-03,Closed with explanation,Yes,No,1732432.0
1782000,2023-02-21,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Credit inquiries on your report that you don't...,NaN,NaN,"EQUIFAX, INC.",TX,76016,NaN,Other,Web,2023-02-21,Closed with explanation,Yes,NaN,6598592.0
1310148,2020-10-30,Credit card or prepaid card,General-purpose credit card or charge card,"Other features, terms, or problems",Problem with balance transfer,NaN,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",FL,32963,NaN,NaN,Referral,2020-10-31,Closed with monetary relief,Yes,NaN,3929739.0
2677066,2022-07-06,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,I recantly went into 2 separate dealerships to...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NC,28214,NaN,Consent provided,Web,2022-07-06,Closed with non-monetary relief,Yes,NaN,5743371.0
2992369,2019-10-16,Credit card or prepaid card,General-purpose credit card or charge card,Closing your account,Company closed your account,"On XX/XX/XXXX, I paid down my Discover balance...",NaN,DISCOVER BANK,PA,19103,NaN,Consent provided,Web,2019-10-16,Closed with explanation,Yes,NaN,3407277.0
2730530,2021-06-16,Mortgage,Conventional home mortgage,Applying for a mortgage or refinancing an exis...,NaN,NaN,NaN,"Rocket Mortgage, LLC",CO,80218,NaN,NaN,Phone,2021-06-16,Closed with explanation,Yes,NaN,4466630.0
1074599,2016-06-22,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt is not mine,I have sent several requests to Pinnacle Credi...,NaN,"Pinnacle Credit Services, LLC",MD,21133,NaN,Consent provided,Web,2016-06-22,Closed,Yes,No,1980972.0
1300253,2021-01-05,Mortgage,Conventional home mortgage,Struggling to pay mortgage,NaN,I am writing to get some help and answers I wa...,Company believes it acted appropriately as aut...,"SELECT PORTFOLIO SERVICING, INC.",NJ,07060,Servicemember,Consent provided,Web,2021-01-05,Closed with explanation,Yes,NaN,4048721.0


In [9]:
# # Create table
# table_creation_query = """
#     CREATE TABLE cfpb (
#         id SERIAL PRIMARY KEY,
#         "Date_received" DATE,
#         "Product" VARCHAR(255),
#         "Sub_product" VARCHAR(255),
#         "Issue" VARCHAR(255),
#         "Sub_issue" VARCHAR(255),
#         "Consumer_complaint_narrative" TEXT,
#         "Company_public_response" VARCHAR(255),
#         "Company" VARCHAR(255),
#         "State" VARCHAR(255),
#         "ZIP_code" VARCHAR(255),
#         "Tags" VARCHAR(255),
#         "Consumer_consent_provided" VARCHAR(255),
#         "Submitted_via" VARCHAR(255),
#         "Date_sent_to_company" DATE,
#         "Company_response_to_consumer" VARCHAR(255),
#         "Timely_response" VARCHAR(255),
#         "Consumer_disputed" VARCHAR(255),
#         "Complaint_ID" INT UNIQUE
#     );
# """

# cur.execute(table_creation_query)
# # Commit your changes
# conn.commit()

In [10]:
%%time
# Convert DataFrame to a list of tuples
records = cfpb.to_records(index=False)
result = list(records)

insert_query_base = """
INSERT INTO cfpb ("Date_received", "Product", "Sub_product", "Issue", "Sub_issue", "Consumer_complaint_narrative", 
"Company_public_response", "Company", "State", "ZIP_code", "Tags", "Consumer_consent_provided", "Submitted_via", 
"Date_sent_to_company", "Company_response_to_consumer", "Timely_response", "Consumer_disputed", "Complaint_ID") 
VALUES %s ON CONFLICT ("Complaint_ID") DO NOTHING
"""
psycopg2.extras.execute_values(cur, insert_query_base, result)

# Commit your changes
conn.commit()

Wall time: 38min 9s


In [11]:
# EXAMPLE USING SIDEBAR MANIPULATING SQL QUERY
# with st.sidebar():
#     state = st.multiselect(options = [states list])



query = f"""
SELECT *
FROM cfpb
WHERE STATES IN ({state})
ORDER BY random()
LIMIT 100;
"""

# # Method 1
# # Fetch data in chunks
# chunk_size = 10000  # you can adjust this value depending on your system's memory and requirements
# chunks = []
# for chunk in pd.read_sql_query(query, conn, chunksize=chunk_size):
#     # process data in chunks as needed, or save each chunk for later processing
#     chunks.append(chunk)

# # Combine chunks into one DataFrame
# df = pd.concat(chunks)


# # Method 2
# # Execute the query
# cur.execute(query)
# # Fetch all rows
# rows = cur.fetchall()
# # Get the column names
# colnames = [desc[0] for desc in cur.description]
# # Convert to DataFrame
# df = pd.DataFrame(rows, columns=colnames)

# Method 3
df = pd.read_sql_query(query, conn)
df

NameError: name 'state' is not defined

In [12]:
# Close the cursor and connection
cur.close()
conn.close()